# Data Engineering

In [92]:
#create environment
import numpy as np
import pandas as pd
import scipy as sci
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from copy import deepcopy
from statsmodels.formula.api import ols
import statsmodels.api as sm
from datetime import datetime
from pandas.plotting import scatter_matrix
pd.set_option('display.max_columns', 500)

### Data

In [93]:
#import data
uno_sale = pd.read_pickle('uno_sale_ec.pkl')
uno_inv = pd.read_pickle('uno_inv_ec.pkl')
uno_sale.shape, uno_inv.shape

((111038, 28), (144612, 7))

In [94]:
def missing_value(df):
    nulls = pd.DataFrame(df.isnull().sum().sort_values(ascending=False))
    nulls.columns=['NAs']
    nulls['percentage']=nulls['NAs']/df.shape[0]
    result=nulls.loc[nulls.NAs>0]
    print('Number of features with NAs:',result.shape[0])
    return result

In [95]:
invqty = uno_inv.groupby(['STYLE#','Description','COLOR']).agg({'LAST RCVD':'count','Qty':'sum', 'On-Hand':'sum'}).sort_values('Qty', ascending=False).reset_index()
invqty['Warehouse'] = invqty['On-Hand'] /invqty['LAST RCVD']
invqty.drop(columns=['LAST RCVD','On-Hand'], inplace=True)
invqty['Discrepancy'] = invqty.Warehouse - invqty.Qty
invqty.head()

STYLE#                 Description  COLOR      Qty  Warehouse  \
0    DTYZ105                     cc bead     GD  10000.0    10000.0   
1  DTA4E2531  teardrop tassel fan  ivory  IVORY   7642.0     7642.0   
2  DTSYC1224         mm metal ball chain   None   6240.0     6240.0   
3  DTSYN2575         metal layered brass     GD   4800.0     4800.0   
4  DTSYC1225         mm metal ball chain   None   3960.0     3960.0   

   Discrepancy  
0          0.0  
1          0.0  
2          0.0  
3          0.0  
4          0.0

In [96]:
invqty['Warehouse'] = invqty['Qty']
invqty.drop(columns=['Discrepancy','Qty'], inplace=True)
invqty.head()

STYLE#                 Description  COLOR  Warehouse
0    DTYZ105                     cc bead     GD    10000.0
1  DTA4E2531  teardrop tassel fan  ivory  IVORY     7642.0
2  DTSYC1224         mm metal ball chain   None     6240.0
3  DTSYN2575         metal layered brass     GD     4800.0
4  DTSYC1225         mm metal ball chain   None     3960.0

In [97]:
missing_value(invqty)

Number of features with NAs: 0


Empty DataFrame
Columns: [NAs, percentage]
Index: []

In [98]:
uno_inv['Sale_Qty'] = np.abs([x if x < 0 else 0 for x in uno_inv.Qty])
uno_inv['Inv_Qty'] = np.abs([x if x > 0 else 0 for x in uno_inv.Qty])
uno_inv.head()

STYLE# COLOR       Description  LAST RCVD    Qty  On-Hand  Categories  \
0  DTA1A9500    IV  seed bead anklet 2016-04-12 -144.0      0.0         NaN   
1  DTA1A9500    IV  seed bead anklet 2016-04-12  144.0      0.0         NaN   
2  DTA1A9500   MUL  seed bead anklet 2016-04-12 -144.0      0.0         NaN   
3  DTA1A9500   MUL  seed bead anklet 2016-04-12  144.0      0.0         NaN   
4  DTA1A9500    TQ  seed bead anklet 2016-04-12 -144.0      0.0         NaN   

   Sale_Qty  Inv_Qty  
0     144.0      0.0  
1       0.0    144.0  
2     144.0      0.0  
3       0.0    144.0  
4     144.0      0.0

In [99]:
uno_inv_agg = uno_inv.groupby(['STYLE#','COLOR','Description','LAST RCVD']).agg('sum').reset_index()
uno_inv_agg.shape

(118598, 9)

In [100]:
uno_inv_agg = pd.merge(uno_inv_agg, invqty, how='left', left_on=['STYLE#','Description','COLOR'],\
                       right_on=['STYLE#','Description','COLOR'])
uno_inv_agg.shape

(118598, 10)

In [101]:
uno_inv_agg.drop(columns=['Qty','On-Hand','Categories'],inplace=True)
uno_inv_agg.head(20)

STYLE#   COLOR                Description  LAST RCVD  Sale_Qty  \
0   DTA1A9500      IV           seed bead anklet 2016-04-12     144.0   
1   DTA1A9500     MUL           seed bead anklet 2016-04-12     144.0   
2   DTA1A9500      TQ           seed bead anklet 2016-04-12     144.0   
3   DTA1B2029  WG-COR   row wood bead stretch br 2016-04-04      12.0   
4   DTA1B2029  WG-COR   row wood bead stretch br 2016-04-06      12.0   
5   DTA1B2029  WG-COR   row wood bead stretch br 2016-04-11       0.0   
6   DTA1B2029  WG-COR   row wood bead stretch br 2016-04-12     240.0   
7   DTA1B2029  WG-COR   row wood bead stretch br 2016-05-20      12.0   
8   DTA1B2029  WG-COR   row wood bead stretch br 2016-06-22      12.0   
9   DTA1B2029  WG-COR   row wood bead stretch br 2016-08-03      12.0   
10  DTA1B2029  WG-COR   row wood bead stretch br 2016-08-22      24.0   
11  DTA1B2029  WG-COR   row wood bead stretch br 2016-08-25       0.0   
12  DTA1B2029  WG-COR   row wood bead stretch br 2017-07-24      48.0   
13  DTA1B2029  WG-CRM  row pearl bead stretch br 2016-04-04      12.0   
14  DTA1B2029  WG-CRM  row pearl bead stretch br 2016-05-25      12.0   
15  DTA1B2029  WG-CRM  row pearl bead stretch br 2016-06-09      24.0   
16  DTA1B2029  WG-CRM  row pearl bead stretch br 2016-08-02      12.0   
17  DTA1B2029  WG-CRM  row pearl bead stretch br 2016-08-03      12.0   
18  DTA1B2029  WG-CRM  row pearl bead stretch br 2016-08-15      12.0   
19  DTA1B2029  WG-CRM  row pearl bead stretch br 2016-08-23      24.0   

    Inv_Qty  Warehouse  
0     144.0        0.0  
1     144.0        0.0  
2     144.0        0.0  
3     120.0        0.0  
4       0.0        0.0  
5     240.0        0.0  
6       0.0        0.0  
7       0.0        0.0  
8       0.0        0.0  
9       0.0        0.0  
10      0.0        0.0  
11     12.0        0.0  
12      0.0        0.0  
13    240.0        0.0  
14      0.0        0.0  
15      0.0        0.0  
16      0.0        0.0  
17      0.0        0.0  
18      0.0        0.0  
19      0.0        0.0

In [102]:
uno_inv_agg.Description = uno_inv_agg.Description.str.lower()
uno_sale.DESCRIPTION = uno_sale.DESCRIPTION.str.lower()

In [103]:
uno_sale_subset = uno_sale.groupby(['DESCRIPTION']).agg({'UNIT PRICE': 'max'}).reset_index()
uno_sale_subset.head()

DESCRIPTION  UNIT PRICE
0                                             8.25
1           a initial bar adj brac  wg        2.75
2           a initial circle disk pend        2.25
3  a large natural srtone in texture l        8.75
4         a linear bar initial pendant        2.75

In [104]:
uno_inv_agg = pd.merge(uno_inv_agg, uno_sale_subset,\
                       how='left', left_on=['Description'],right_on=['DESCRIPTION'])

STYLE#   COLOR               Description  LAST RCVD  Sale_Qty  Inv_Qty  \
0  DTA1A9500      IV          seed bead anklet 2016-04-12     144.0    144.0   
1  DTA1A9500     MUL          seed bead anklet 2016-04-12     144.0    144.0   
2  DTA1A9500      TQ          seed bead anklet 2016-04-12     144.0    144.0   
3  DTA1B2029  WG-COR  row wood bead stretch br 2016-04-04      12.0    120.0   
4  DTA1B2029  WG-COR  row wood bead stretch br 2016-04-06      12.0      0.0   

   Warehouse               DESCRIPTION  UNIT PRICE  
0        0.0          seed bead anklet        4.00  
1        0.0          seed bead anklet        4.00  
2        0.0          seed bead anklet        4.00  
3        0.0  row wood bead stretch br        3.25  
4        0.0  row wood bead stretch br        3.25

In [113]:
#uno_inv_agg.drop(columns='DESCRIPTION', inplace=True)
uno_inv_agg.head(10)

STYLE#   COLOR               Description  LAST RCVD  Sale_Qty  Inv_Qty  \
0  DTA1A9500      IV          seed bead anklet 2016-04-12     144.0    144.0   
1  DTA1A9500     MUL          seed bead anklet 2016-04-12     144.0    144.0   
2  DTA1A9500      TQ          seed bead anklet 2016-04-12     144.0    144.0   
3  DTA1B2029  WG-COR  row wood bead stretch br 2016-04-04      12.0    120.0   
4  DTA1B2029  WG-COR  row wood bead stretch br 2016-04-06      12.0      0.0   
5  DTA1B2029  WG-COR  row wood bead stretch br 2016-04-11       0.0    240.0   
6  DTA1B2029  WG-COR  row wood bead stretch br 2016-04-12     240.0      0.0   
7  DTA1B2029  WG-COR  row wood bead stretch br 2016-05-20      12.0      0.0   
8  DTA1B2029  WG-COR  row wood bead stretch br 2016-06-22      12.0      0.0   
9  DTA1B2029  WG-COR  row wood bead stretch br 2016-08-03      12.0      0.0   

   Warehouse  UNIT PRICE  
0        0.0        4.00  
1        0.0        4.00  
2        0.0        4.00  
3        0.0        3.25  
4        0.0        3.25  
5        0.0        3.25  
6        0.0        3.25  
7        0.0        3.25  
8        0.0        3.25  
9        0.0        3.25

In [110]:
uno_inv_agg.shape

(118598, 8)

In [111]:
missing_value(uno_inv_agg) #7286 items were never sold/left warehouse

Number of features with NAs: 1


NAs  percentage
UNIT PRICE  7286    0.061434

In [127]:
temp = uno_inv_agg[uno_inv_agg.isnull().any(axis=1)]
temp.loc[temp["Warehouse"]>0,:]

STYLE#    COLOR                  Description  LAST RCVD  \
4180         DTA1N2029   WG-MST  wood bead body  tassel   nk 2016-04-11   
4181         DTA1N2029   WG-MST  wood bead body  tassel   nk 2016-05-17   
4182         DTA1N2029   WG-MST  wood bead body  tassel   nk 2016-06-15   
4183         DTA1N2029   WG-MST  wood bead body  tassel   nk 2017-05-25   
4184         DTA1N2029   WG-MST  wood bead body  tassel   nk 2018-01-17   
4185         DTA1N2029   WG-MST  wood bead body  tassel   nk 2018-04-27   
4186         DTA1N2029   WG-MST  wood bead body  tassel   nk 2018-05-03   
4187         DTA1N2029   WG-MST  wood bead body  tassel   nk 2018-07-27   
21191        DTA4B2546      CML    magnetic pu  be brave  br 2017-02-02   
21192        DTA4B2546      CML    magnetic pu  be brave  br 2017-02-03   
21193        DTA4B2546      CML    magnetic pu  be brave  br 2017-02-07   
21194        DTA4B2546      CML    magnetic pu  be brave  br 2017-02-17   
21195        DTA4B2546      CML    magnetic pu  be brave  br 2017-03-01   
21196        DTA4B2546      CML    magnetic pu  be brave  br 2017-03-03   
21197        DTA4B2546      CML    magnetic pu  be brave  br 2017-03-08   
21198        DTA4B2546      CML    magnetic pu  be brave  br 2017-05-24   
21199        DTA4B2546      CML    magnetic pu  be brave  br 2017-10-25   
21200        DTA4B2546      CML    magnetic pu  be brave  br 2017-11-28   
21201        DTA4B2546      JET       magnetic pu  faith  br 2017-02-02   
21202        DTA4B2546      JET       magnetic pu  faith  br 2017-02-03   
21203        DTA4B2546      JET       magnetic pu  faith  br 2017-02-07   
21204        DTA4B2546      JET       magnetic pu  faith  br 2017-02-17   
21205        DTA4B2546      JET       magnetic pu  faith  br 2017-03-01   
21206        DTA4B2546      JET       magnetic pu  faith  br 2017-03-03   
21207        DTA4B2546      JET       magnetic pu  faith  br 2017-03-08   
21208        DTA4B2546      JET       magnetic pu  faith  br 2017-03-15   
21209        DTA4B2546      JET       magnetic pu  faith  br 2017-05-09   
21210        DTA4B2546      JET       magnetic pu  faith  br 2017-08-30   
21211        DTA4B2546      JET       magnetic pu  faith  br 2017-11-28   
21223       DTA4B2546F      TAN    magnetic pu  fearless  br 2017-02-02   
...                ...      ...                          ...        ...   
118314    DTYLN-805011       NV           chain w tassel  dn 2018-07-16   
118315    DTYLN-805011       NV           chain w tassel  dn 2018-08-27   
118316    DTYLN-805011       NV           chain w tassel  dn 2018-09-18   
118322    DTYLN-807016     ADOB    layered crysltort pend nk 2018-11-07   
118323    DTYLN-807016   TANNIN    layered crysltort pend nk 2018-11-07   
118324    DTYLN-807038       GD      metal chain w mm o ring 2018-08-31   
118325    DTYLN-807038       GD      metal chain w mm o ring 2018-10-02   
118326    DTYLN-807038       GD      metal chain w mm o ring 2018-10-15   
118329    DTYLN-807066       GD      metal chain w mm o ring 2018-08-31   
118330    DTYLN-807067       GD      metal chain w mm o ring 2018-08-20   
118333    DTYLN-808005       WG       faith mini pendant  wg 2018-11-21   
118355    DTYLP-707001      AMY           fabric tassel  amy 2017-09-25   
118356    DTYLP-707001      GRY          fabric tassel   gry 2017-09-25   
118357  DTYLP-707002-3      JET                dn tassel jet 2018-10-02   
118374  DTYLP-801003-2  JET-GRY               layered tassel 2018-05-29   
118375  DTYLP-801003-2  JET-GRY               layered tassel 2018-07-12   
118376  DTYLP-801003-6   FU-DPK               layered tassel 2018-05-29   
118377  DTYLP-801003-6   FU-DPK               layered tassel 2018-07-12   
118378  DTYLP-801003-7      MUL               layered tassel 2018-05-29   
118379  DTYLP-801003-7      MUL               layered tassel 2018-07-12   
118380  DTYLP-801003-8  IV-LGRY               layered tassel 2018-05-29   
118381  DTYLP-801003-8  IV-LGRY          

Number of features with NAs: 2


NAs  percentage
UNIT PRICE   184954    0.001173
DESCRIPTION  184954    0.001173

STYLE#                0
COLOR                 0
DESCRIPTION           0
CUST #                0
INVC DATE             0
QTY                   0
UNIT PRICE            0
EXT-AMT               0
REASON                0
IMPORT HANDMADE       0
MANUFACTORS           0
State_sold            0
ZIP_sold              0
RELEASED YEAR         0
Seasos                0
Season target         0
LAST RCVD             0
FIRST RCVD            0
LAST INVOICED         0
FIRST INVOICED        0
Categories            0
Categories1           0
Attribute             0
material_style        0
year                  0
month                 0
Style_Color        3981
Revenue               0
dtype: int64